# log
- logging
- logging模块提供模块级别的函数记录日志
- 包括四大组件

## 日志相关概念
- 日志
- 日志的级别(level)
	- 不同的用户关注不同的程序信息
- IO操作 => 不要太频繁
- LOG的作用
	- 调试
	- 了解软件的运行情况
	- 分析定位问题
- 日志信息
	- time
	- 地点
	- level
	- 内容
- 成熟的第三方日志
	- log4j
	- log4php
	- logging


# Logging模块
- 日志级别
	- 级别可自定义
- 初始化/写日志实例需要指定级别, 只有当级别等于或高于指定级别才被记录
- 使用方式
	- 直接使用logging(封装了其他组件)
	- Logging四大组件直接定制


# logging模块级别的日志
- 使用以下几个函数	
	- logging.debug(msg, *args, **kwargs)	   创建一条严重级别为DEBUG的日志记录
	- logging.info(msg, *args, **kwargs)	    创建一条严重级别为INFO的日志记录
	- logging.warning(msg, *args, **kwargs)	 创建一条严重级别为WARNING的日志记录
	- logging.error(msg, *args, **kwargs)	   创建一条严重级别为ERROR的日志记录
	- logging.critical(msg, *args, **kwargs)	创建一条严重级别为CRITICAL的日志记录
	- logging.log(level, *args, **kwargs)	   创建一条严重级别为level的日志记录
	- logging.basicConfig(**kwargs)	         对root logger进行一次性配置
- logging.basicConfig(**kwargs)	对root logger进行一次性配置
	- 只在第一次调用的时候起作用
	- 不配置logger则使用默认值
		- 输出: sys.stderr
		- 级别: WARNING
		- 格式: level:log_name:content

In [3]:
import logging
logging.basicConfig(filename='tulingxueyuan.txt', level=logging.DEBUG)

logging.debug("This is a debug log.")
logging.info("This is a info log.")
logging.warning("This is a warning log.")
logging.error("This is a error log.")
logging.critical("This is a critical log.")
# 另一种写法
# logging.log(logging.DEBUG, "This is a debug log.")
# logging.log(logging.INFO, "This is a info log.")
# logging.log(logging.WARNING, "This is a warning log.")
# logging.log(logging.ERROR, "This is a error log.")
# logging.log(logging.CRITICAL, "This is a critical log.")

# logging模块的处理流程
- 四大组件
	- 日志器(logger):产生日志的一个接口
	- 处理器(handler):把产生的日志发送到相应的目的地
	- 过滤器(filter):更精细的控制哪些日志输出
	- 格式器(formatter):对输出信息进行格式化
- Logger
	- 产生一个日志
	- 操作
		logger.setLevel()	设置日志器将会处理的日志消息的最低严重级别
		logger.addHandler() 和 logger.removeHandler()   为该logger对象添加 和 移除一个Handler()
		logger.addFilter() 和 logger.removeFilter()  为该logger对象添加 和 移除一个filter()
		logger.debug: 产生一条debug级别的日志,同理,info,error...等
		logger.exception(): 创建类似于logger.error的日志消息
		logger.log():  获取一个明确的日志level参数类创建一个日志记录
	- 如何得到一个logger对象
		- 实例化
		- logging.getLogger()
- Handler
	- 把log发送到指定位置
	- 方法
		- setLevel
		- setFormat
		- addFilter, removeFilter
	- 不需要直接使用,Handler是基类
				Handler			描述
		logging.StreamHandler			将日志消息发送到输出到Stream，如std.out, std.err或任何file-like对象。
		logging.FileHandler			将日志消息发送到磁盘文件，默认情况下文件大小会无限增长
		logging.handlers.RotatingFileHandler	将日志消息发送到磁盘文件，并支持日志文件按大小切割
		logging.hanlders.TimedRotatingFileHandler	将日志消息发送到磁盘文件，并支持日志文件按时间切割
		logging.handlers.HTTPHandler		将日志消息以GET或POST的方式发送给一个HTTP服务器
		logging.handlers.SMTPHandler		将日志消息发送给一个指定的email地址
		logging.NullHandler	该Handler实例会忽略error messages，通常被想使用logging的library开发者使用来避免'No handlers could be found for logger XXX'信息的出现。
- Format类
	- 直接实例化
	- 可以继承Format添加特殊内容
	- 三个参数
		fmt：指定消息格式化字符串，如果不指定该参数则默认使用message的原始值
		datefmt：指定日期格式字符串，如果不指定该参数则默认使用"%Y-%m-%d %H:%M:%S"
		style：Python 3.2新增的参数，可取值为 '%', '{'和 '$'，如果不指定该参数则默认使用'%'
- Filter类
	- 可以被Handler和logger使用
	- 控制传递过来的信息的具体内容

In [15]:
# 1. 需求
# 现在有以下几个日志记录的需求：
#     1）要求将所有级别的所有日志都写入磁盘文件中
#     2）all.log文件中记录所有的日志信息，日志格式为：日期和时间 - 日志级别 - 日志信息
#     3）error.log文件中单独记录error及以上级别的日志信息，日志格式为：日期和时间 - 日志级别 - 文件名[:行号] - 日志信息
#     4）要求all.log在每天凌晨进行日志切割

# 2. 分析
#     1）要记录所有级别的日志，因此日志器的有效level需要设置为最低级别--DEBUG;
#     2）日志需要被发送到两个不同的目的地，因此需要为日志器设置两个handler；另外，两个目的地都是磁盘文件，因此这两个handler都是与FileHandler相关的；
#     3）all.log要求按照时间进行日志切割，因此他需要用logging.handlers.TimedRotatingFileHandler; 而error.log没有要求日志切割，因此可以使用FileHandler;
#     4）两个日志文件的格式不同，因此需要对这两个handler分别设置格式器；